In [1]:
# Load data from /Volumes/Exodus/Data/6DOF 2023/Test1

import os
import pandas as pd

# Sample data from a text file
""" 
Time (ms)	169448
Reference	-112.9437	180.4520	-208.2180	0.4736	0.4635	-0.5261	0.5330
Fenestrated	-21.9398	56.9237	-295.1500	0.0451	0.2661	-0.8368	-0.4763
Curved	-62.3648	51.9654	-243.1492	0.2314	0.3947	-0.7884	0.4112
Camera	-75.6810	35.3394	-270.7886	0.5233	-0.3824	-0.6371	-0.4171
"""


# Define the function to extract data from text files and create DataFrames for each tool
def extract_data_from_txt_files(directory):
    data = {"Reference": [], "Fenestrated": [], "Curved": [], "Camera": []}

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            try:
                with open(filepath, "r") as file:
                    lines = file.readlines()
                    # Store time in milliseconds
                    time = int(lines[0].split()[2])
                    for line in lines[1:]:  # Skip the header
                        parts = line.strip().split()
                        if len(parts) == 8:
                            label = parts[0]
                            x, y, z = float(parts[1]), float(parts[2]), float(parts[3])
                            qx, qy, qz, qw = (
                                float(parts[4]),
                                float(parts[5]),
                                float(parts[6]),
                                float(parts[7]),
                            )
                            data[label].append(
                                {
                                    "time": time,
                                    "x": x,
                                    "y": y,
                                    "z": z,
                                    "qx": qx,
                                    "qy": qy,
                                    "qz": qz,
                                    "qw": qw,
                                }
                            )
            except Exception as e:
                print(f"Error reading {filepath}: {e}")
    return data

Error reading /Volumes/Exodus/Data/6DOF 2023/Test 1/._4212.txt: 'utf-8' codec can't decode byte 0xb0 in position 37: invalid start byte
      time         x         y         z      qx      qy      qz      qw
2980     0 -112.9454  180.4771 -208.2749  0.4739  0.4630 -0.5262  0.5330
3113    33 -112.9293  180.4841 -208.2801  0.4739  0.4629 -0.5263  0.5330
2840    64 -112.9255  180.4887 -208.2706  0.4739  0.4629 -0.5263  0.5330
2717    97 -112.9348  180.4809 -208.2788  0.4739  0.4629 -0.5263  0.5330
2296   129 -112.9364  180.4964 -208.2635  0.4740  0.4629 -0.5264  0.5329
      time        x        y         z      qx      qy      qz      qw
2980     0 -88.5983  70.8865 -302.3875  0.8246  0.1419  0.3515  0.4199
3113    33 -89.0880  69.9984 -302.1611  0.8241  0.1425  0.3511  0.4211
2840    64 -88.6904  70.5903 -302.9671  0.8249  0.1423  0.3508  0.4198
2717    97 -88.6753  70.5547 -302.8969  0.8251  0.1428  0.3501  0.4198
2296   129 -87.2999  72.5254 -304.2392  0.8273  0.1414  0.3499  0.4161


In [2]:
PATH_6DOF = "data/6DOF/"  # Path to CSV data
DATA_PATH = "/Volumes/Exodus/Data/6DOF 2023/"  # Local raw data path

# Store folders which exist inside DATA_PATH
DIRECTORIES = [f for f in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, f))]

In [ ]:
# Create all data files
for directory in DIRECTORIES:
    if not os.path.exists(PATH_6DOF + directory):
        os.makedirs(PATH_6DOF + directory)
        directory_path = DATA_PATH + directory + "/"
        extracted_data = extract_data_from_txt_files(directory_path)

        # Convert the extracted data into pandas DataFrames
        reference_df = pd.DataFrame(extracted_data["Reference"]).sort_values("time")
        fenestrated_df = pd.DataFrame(extracted_data["Fenestrated"]).sort_values("time")
        curved_df = pd.DataFrame(extracted_data["Curved"]).sort_values("time")
        camera_df = pd.DataFrame(extracted_data["Camera"]).sort_values("time")

        reference_df.to_csv(PATH_6DOF + directory + "/reference.csv", index=False)
        fenestrated_df.to_csv(PATH_6DOF + directory + "/fenestrated.csv", index=False)
        curved_df.to_csv(PATH_6DOF + directory + "/curved.csv", index=False)
        camera_df.to_csv(PATH_6DOF + directory + "/camera.csv", index=False)
        print(f"Created data files for {directory}")
    else:
        print(f"Data files for {directory} already exist")

    # # Displaying the first few entries for verification
    # # print(reference_df.head())
    # # print(fenestrated_df.head())
    # # print(curved_df.head())
    # # print(camera_df.head())

    # # Displaying the first few entries for verification
    # print(reference_df.shape)
    # print(fenestrated_df.shape)
    # print(curved_df.shape)
    # print(camera_df.shape)

print("All data files created")

In [3]:
# Load the data from the CSV files
def load_data(directory):
    reference_df = pd.read_csv(PATH_6DOF + directory + "/reference.csv")
    fenestrated_df = pd.read_csv(PATH_6DOF + directory + "/fenestrated.csv")
    curved_df = pd.read_csv(PATH_6DOF + directory + "/curved.csv")
    camera_df = pd.read_csv(PATH_6DOF + directory + "/camera.csv")
    return reference_df, fenestrated_df, curved_df, camera_df

reference_df, fenestrated_df, curved_df, camera_df = load_data("Test1")

In [ ]:
# Custom function which will run for all the directories
def load_all_data():
    all_data = {}
    for directory in DIRECTORIES:
        reference_df, fenestrated_df, curved_df, camera_df = load_data(directory)
        all_data[directory] = {
            "Reference": reference_df,
            "Fenestrated": fenestrated_df,
            "Curved": curved_df,
            "Camera": camera_df,
        }
    return all_data

# data = load_all_data()

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Define a function to create scatter plots for x, y, and z coordinates
def create_2d_scatter_plots(df, tool_name):
    fig = make_subplots(
        rows=1, cols=3, subplot_titles=("X Coordinate", "Y Coordinate", "Z Coordinate")
    )

    fig.add_trace(
        go.Scatter(x=df["time"], y=df["x"], mode="markers", name="x"), row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=df["time"], y=df["y"], mode="markers", name="y"), row=1, col=2
    )

    fig.add_trace(
        go.Scatter(x=df["time"], y=df["z"], mode="markers", name="z"), row=1, col=3
    )

    fig.update_layout(
        title_text=f"2D Scatter Plots of {tool_name} Tool Coordinates Over Time"
    )
    fig.show()


# Plot the data for each tool
create_2d_scatter_plots(reference_df, "Reference")
create_2d_scatter_plots(fenestrated_df, "Fenestrated")
create_2d_scatter_plots(curved_df, "Curved")
create_2d_scatter_plots(camera_df, "Camera")

In [5]:
# Define a function to create a combined 3D motion plot for all tools
def create_combined_3d_motion_plot(dfs, tool_names):
    fig = make_subplots(
        rows=1,
        cols=4,
        specs=[
            [
                {"type": "scatter3d"},
                {"type": "scatter3d"},
                {"type": "scatter3d"},
                {"type": "scatter3d"},
            ]
        ],
        subplot_titles=tool_names,
    )

    for i, (df, tool_name) in enumerate(zip(dfs, tool_names), start=1):
        fig.add_trace(
            go.Scatter3d(
                x=df["x"],
                y=df["y"],
                z=df["z"],
                mode="lines+markers",
                marker=dict(size=4),
                line=dict(width=2),
                name=tool_name,
            ),
            row=1,
            col=i,
        )

    fig.update_layout(
        title="3D Motion Plots of Tools",
        scene=dict(
            xaxis_title="X Position", yaxis_title="Y Position", zaxis_title="Z Position"
        ),
    )

    fig.show()

# DataFrames for each tool
dfs = [reference_df, fenestrated_df, curved_df, camera_df]
tool_names = ["Reference", "Fenestrated", "Curved", "Camera"]

# Create the combined 3D motion plot
create_combined_3d_motion_plot(dfs, tool_names)

In [6]:
import numpy as np
import imageio
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

total_frames = max(len(df) for df in [reference_df, fenestrated_df, curved_df, camera_df])

# Only take first 10 rows for animation
# reference_df = reference_df.head(10)
# fenestrated_df = fenestrated_df.head(10)
# curved_df = curved_df.head(10)
# camera_df = camera_df.head(10)

In [ ]:
# Preprocess the DataFrame to add a frame column
def preprocess_for_animation(df):
    df = df.copy()
    df["frame"] = np.arange(len(df))
    return df


reference_df = preprocess_for_animation(reference_df)
fenestrated_df = preprocess_for_animation(fenestrated_df)
curved_df = preprocess_for_animation(curved_df)
camera_df = preprocess_for_animation(camera_df)

In [ ]:
# Calculate the min and max values for each axis across all tools
x_min = min(
    reference_df["x"].min(),
    fenestrated_df["x"].min(),
    curved_df["x"].min(),
    camera_df["x"].min(),
)
x_max = max(
    reference_df["x"].max(),
    fenestrated_df["x"].max(),
    curved_df["x"].max(),
    camera_df["x"].max(),
)
y_min = min(
    reference_df["y"].min(),
    fenestrated_df["y"].min(),
    curved_df["y"].min(),
    camera_df["y"].min(),
)
y_max = max(
    reference_df["y"].max(),
    fenestrated_df["y"].max(),
    curved_df["y"].max(),
    camera_df["y"].max(),
)
z_min = min(
    reference_df["z"].min(),
    fenestrated_df["z"].min(),
    curved_df["z"].min(),
    camera_df["z"].min(),
)
z_max = max(
    reference_df["z"].max(),
    fenestrated_df["z"].max(),
    curved_df["z"].max(),
    camera_df["z"].max(),
)

In [ ]:
# Function to create 3D plots and save as images
def create_3d_plots(dfs, tool_names, save_path, dpi=300, figsize=(10, 10)):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for k in range(total_frames):
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(111, projection="3d")

        for df, tool_name in zip(dfs, tool_names):
            if k < len(df):
                x_vals = df["x"][: k + 1]
                y_vals = df["y"][: k + 1]
                z_vals = df["z"][: k + 1]

                if len(x_vals) >= 3:
                    ax.plot(
                        x_vals[-3:],
                        y_vals[-3:],
                        z_vals[-3:],
                        label=tool_name,
                        alpha=0.2,
                    )
                if len(x_vals) >= 2:
                    ax.plot(
                        x_vals[-2:],
                        y_vals[-2:],
                        z_vals[-2:],
                        label=tool_name,
                        alpha=0.4,
                    )
                if len(x_vals) >= 1:
                    ax.plot(
                        x_vals[-1:],
                        y_vals[-1:],
                        z_vals[-1:],
                        label=tool_name,
                        alpha=1.0,
                    )

        ax.set_xlim([x_min, x_max])
        ax.set_ylim([y_min, y_max])
        ax.set_zlim([z_min, z_max])
        ax.set_xlabel("X Position")
        ax.set_ylabel("Y Position")
        ax.set_zlabel("Z Position")
        ax.set_title("3D Motion of Tools")

        # Plot unique label names only
        handles, labels = ax.get_legend_handles_labels()
        unique_labels = []
        unique_handles = []
        for i, label in enumerate(labels):
            if label not in unique_labels:
                unique_labels.append(label)
                unique_handles.append(handles[i])
        ax.legend(unique_handles, unique_labels)

        plt.savefig(f"{save_path}/frame_{k:04d}.png", dpi=dpi, bbox_inches="tight")
        plt.close()

        # print(f"Generated frame {k}/{total_frames}")


# Create the 3D plots and save frames
save_path = "frames"
dfs = [reference_df, fenestrated_df, curved_df, camera_df]
tool_names = ["Reference", "Fenestrated", "Curved", "Camera"]

# If no frames exist then create them
if not os.path.exists(save_path):
    os.makedirs(save_path)
    create_3d_plots(dfs, tool_names, save_path, dpi=300, figsize=(10, 10))
else:
    print("Frames already exist")

In [ ]:
fixed_size = (2400, 2448)

# Resize images to fixed size
for image in os.listdir(save_path):
    if image.endswith(".png") and not image.startswith("resized_"):
        img_path = os.path.join(save_path, image)
        img = Image.open(img_path)
        img = img.resize(
            fixed_size, Image.Resampling.LANCZOS
        )  # Use Image.Resampling.LANCZOS
        img.save(os.path.join(save_path, "resized_" + image))
        # Delete original image
        os.remove(img_path)

In [ ]:
# Function to create a video from images
def create_video_from_images(image_folder, output_video_path, fps=10):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images.sort()
    frame = Image.open(os.path.join(image_folder, images[0]))
    frame_width, frame_height = frame.size

    writer = imageio.get_writer(output_video_path, fps=fps)
    for image in images:
        img_path = os.path.join(image_folder, image)
        writer.append_data(imageio.imread(img_path))
    writer.close()


# Create a video from the saved frames
output_video_path = "motion_video_6DOF.mp4"
if not os.path.exists(output_video_path):
    create_video_from_images(save_path, output_video_path, fps=10)
else:
    print("Video already exists")